# Manejo de datos scrapeados de Sklearn user guide

Estudio de los datos scrapeados de la guia de usuario de Scikit-learn [https://scikit-learn.org/stable/user_guide.html](https://scikit-learn.org/stable/user_guide.html)

Cargo los datos de un archivo pickle previamente descargado por un script de scrapping en Python.

In [5]:
import pickle

with open('sklearn_guide.plk','rb') as rick:
    df_guide = pickle.load(rick)

df_guide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   level1   297 non-null    object
 1   level2   297 non-null    object
 2   level3   297 non-null    object
 3   content  297 non-null    object
dtypes: object(4)
memory usage: 9.4+ KB


In [7]:
df_guide.iloc[-1]

level1             Common pitfalls and recommended practices
level2                                controlling-randomness
level3                                                      
content    Some scikit-learn objects are inherently rando...
Name: 296, dtype: object